In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Stomach_Cancer"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Stomach_Cancer/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Stomach_Cancer/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Stomach_Cancer/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Stomach_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# Step 1: Search for directories related to Stomach Cancer
import os

# List all directories in TCGA root directory
tcga_dirs = os.listdir(tcga_root_dir)

# Look for stomach cancer datasets
matching_dirs = [dir_name for dir_name in tcga_dirs 
                if any(term in dir_name.lower() for term in 
                       ["stomach", "gastric"])]

if not matching_dirs:
    print(f"No exact matching directory found for trait: {trait}")
    print(f"Available directories: {tcga_dirs}")
    
    # Record that this trait is not available and exit
    validate_and_save_cohort_info(
        is_final=False,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=False,
        is_trait_available=False
    )
    print(f"Task marked as completed. {trait} is not directly represented in the TCGA dataset.")
else:
    # If we found matching directories
    print(f"Found matching directories: {matching_dirs}")
    
    # Select the appropriate directory for stomach cancer
    selected_dir = matching_dirs[0]  # Default to first match
    if "TCGA_Stomach_Cancer_(STAD)" in matching_dirs:
        selected_dir = "TCGA_Stomach_Cancer_(STAD)"  # This is the exact match
    
    print(f"Selected directory: {selected_dir}")
    cohort_dir = os.path.join(tcga_root_dir, selected_dir)
    
    # Step 2: Get file paths for clinical and genetic data
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    # Step 3: Load the files
    clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
    genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)
    
    # Step 4: Print column names of clinical data
    print("Clinical data columns:")
    print(clinical_df.columns.tolist())


Found matching directories: ['TCGA_Stomach_Cancer_(STAD)']
Selected directory: TCGA_Stomach_Cancer_(STAD)


Clinical data columns:
['CDE_ID_3226963', '_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'additional_surgery_metastatic_procedure', 'age_at_initial_pathologic_diagnosis', 'anatomic_neoplasm_subdivision', 'antireflux_treatment', 'antireflux_treatment_type', 'barretts_esophagus', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'city_of_procurement', 'country_of_procurement', 'days_to_additional_surgery_locoregional_procedure', 'days_to_additional_surgery_metastatic_procedure', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'family_history_of_stomach_cancer', 'followup_case_report_form_submission_reason', 'followup_treatment_success', 'form_completion_date', 'gender', 'h_pylori_infection', 'histological_t

### Step 2: Find Candidate Demographic Features

In [3]:
# 1. Identify age and gender columns from clinical data columns
candidate_age_cols = [
    'age_at_initial_pathologic_diagnosis', 
    'days_to_birth'
]

candidate_gender_cols = [
    'gender'
]

# 2. Extract and preview the candidate columns
# First, we need to load the clinical data
cohort_dir = os.path.join(tcga_root_dir, 'TCGA_Stomach_Cancer_(STAD)')
clinical_file_path, _ = tcga_get_relevant_filepaths(cohort_dir)

# Try reading with different delimiters with error handling
try:
    # First try tab delimiter which is common in TCGA files
    clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep='\t')
except Exception as e:
    try:
        # Try with automatic delimiter detection
        clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep=None, engine='python')
    except Exception as e:
        print(f"Error reading clinical file: {e}")
        # Create an empty DataFrame if all attempts fail
        clinical_df = pd.DataFrame()

# Extract candidate age columns
if not clinical_df.empty:
    age_preview = {}
    for col in candidate_age_cols:
        if col in clinical_df.columns:
            age_preview[col] = clinical_df[col].head(5).tolist()
    
    print("Age columns preview:")
    print(age_preview)

    # Extract candidate gender columns
    gender_preview = {}
    for col in candidate_gender_cols:
        if col in clinical_df.columns:
            gender_preview[col] = clinical_df[col].head(5).tolist()
    
    print("Gender columns preview:")
    print(gender_preview)
else:
    print("Could not load clinical data file.")


Age columns preview:
{'age_at_initial_pathologic_diagnosis': [70.0, 51.0, 51.0, 62.0, 52.0], 'days_to_birth': [nan, nan, -18698.0, -22792.0, -19014.0]}
Gender columns preview:
{'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}


### Step 3: Select Demographic Features

In [4]:
# Evaluating the age columns
age_cols = {
    'age_at_initial_pathologic_diagnosis': [70.0, 51.0, 51.0, 62.0, 52.0], 
    'days_to_birth': [float('nan'), float('nan'), -18698.0, -22792.0, -19014.0]
}

# Evaluating the gender columns
gender_cols = {
    'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']
}

# Selecting columns for demographic information
# For age, 'age_at_initial_pathologic_diagnosis' appears better as it has no missing values in the preview
# and directly represents age in years
age_col = 'age_at_initial_pathologic_diagnosis'

# For gender, there's only one column and it has valid values
gender_col = 'gender'

# Print chosen columns
print(f"Chosen age column: {age_col}")
print(f"Age column preview: {age_cols[age_col]}")
print(f"Chosen gender column: {gender_col}")
print(f"Gender column preview: {gender_cols[gender_col]}")


Chosen age column: age_at_initial_pathologic_diagnosis
Age column preview: [70.0, 51.0, 51.0, 62.0, 52.0]
Chosen gender column: gender
Gender column preview: ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize clinical features
# Use the Stomach Cancer directory identified in Step 1
selected_dir = "TCGA_Stomach_Cancer_(STAD)"
cohort_dir = os.path.join(tcga_root_dir, selected_dir)

# Get the file paths for clinical and genetic data
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

# Load the data
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)

# Extract standardized clinical features using the provided trait variable
clinical_features = tcga_select_clinical_features(
    clinical_df, 
    trait=trait,  # Using the provided trait variable
    age_col=age_col, 
    gender_col=gender_col
)

# Save the clinical data to out_clinical_data_file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Saved clinical data with {len(clinical_features)} samples")

# Step 2: Normalize gene symbols in gene expression data
# Transpose to get genes as rows
gene_df = genetic_df

# Normalize gene symbols using NCBI Gene database synonyms
normalized_gene_df = normalize_gene_symbols_in_index(gene_df)
print(f"After normalization: {len(normalized_gene_df)} genes remaining")

# Save the normalized gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Saved normalized gene expression data")

# Step 3: Link clinical and genetic data
# Merge clinical features with genetic expression data
linked_data = clinical_features.join(normalized_gene_df.T, how='inner')
print(f"Linked data shape: {linked_data.shape} (samples x features)")

# Step 4: Handle missing values
cleaned_data = handle_missing_values(linked_data, trait_col=trait)
print(f"After handling missing values, data shape: {cleaned_data.shape}")

# Step 5: Determine if trait or demographics are severely biased
trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait=trait)

# Step 6: Validate data quality and save cohort information
note = "The dataset contains gene expression data along with clinical information for stomach cancer patients from TCGA."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=trait_biased,
    df=cleaned_data,
    note=note
)

# Step 7: Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Saved usable linked data to {out_data_file}")
else:
    print("Dataset was determined to be unusable and was not saved.")

Saved clinical data with 580 samples
After normalization: 19848 genes remaining


Saved normalized gene expression data
Linked data shape: (450, 19851) (samples x features)


After handling missing values, data shape: (450, 19851)
For the feature 'Stomach_Cancer', the least common label is '0' with 35 occurrences. This represents 7.78% of the dataset.
The distribution of the feature 'Stomach_Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 58.0
  50% (Median): 67.0
  75%: 73.0
Min: 30.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 159 occurrences. This represents 35.33% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Saved usable linked data to ../../output/preprocess/Stomach_Cancer/TCGA.csv
